In [ ]:
import re
import pandas as pd
from konlpy.tag import Okt

# 1. 파일 로드 및 기본 전처리


file_path = "/content/금융취약계층.xlsx"
df = pd.read_excel(file_path)
df['카테고리'] = "금융 취약계층"

# 기본 전처리 함수들
def convert_date_format(df):
    df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d', errors='coerce')
    return df

def remove_duplicates(df):
    return df.drop_duplicates(subset=['제목', 'URL'], keep='first')

def filter_unwanted_articles(df):
    unwanted_keywords = ["인사", "공지", "광고", "채용", "발표"]
    return df[~df['제목'].str.contains('|'.join(unwanted_keywords), na=False)]

def handle_missing_values(df):
    return df.fillna("")

# 기본 전처리 적용
df = convert_date_format(df)
df = remove_duplicates(df)
df = filter_unwanted_articles(df)
df = handle_missing_values(df)
print("✅ 기본 전처리 후 데이터 상태:")
print(df.info())
print(df.head(), "\n")

# 2. 추가 전처리: 텍스트 정제, 토큰화, NER 적용

# (1) 텍스트 정제: HTML 태그 및 특수문자 제거
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'<.*?>', '', text)  
    text = re.sub(r'[^\w\s]', '', text) 
    return text

# (2) 형태소 분석 및 불용어 제거 (Okt 사용)
okt = Okt()
stopwords = set(["의", "가", "이", "은", "들", "는", "좀", "잘", "걍", "과", "도", "를", "으로", "자", "에", "와", "한", "하다"])

def preprocess_text(text):
    text = clean_text(text)
    tokens = [token for token in okt.nouns(text) if len(token) > 1]
    tokens = [t for t in tokens if t not in stopwords]
    return tokens

# (3) 사전 기반 NER: 공기업 및 정책 관련 엔터티 추출
public_company_list = [
    "산업은행", "한국수출입은행", "신용보증기금", "기술보증기금",
    "서민금융진흥원", "한국주택금융공사", "한국자산관리공사",
    "중소기업진흥공단", "한국예탁결제원", "금융감독원", "기업은행",
    "신보", "예보", "금융공기업", "국민은행", "ibk기업은행", "우리은행",
    "신한은행", "KEB하나은행", "SC제일은행", "외환은행", "한국시티은행",
    "iM뱅크", "농협은행", "수협", "경남은행", "광주은행", "부산은행",
    "제주은행", "전북은행", "대구은행"  ]

# 정책 관련 키워드
policy_keywords = [
    "청년 대출 지원", "금융지원", "대출 지원", "보증 지원", "정책",
    "금융정책", "지원", "교육", "금융 접근성", "금융 소외", "금융 포용",
    "저소득층", "서민금융",
    "사회적 금융", "금융 혁신", "금융개혁", "금융 규제 완화", "금융 안정",
    "금융 융합", "금융 소비자 보호", "신용회복", "금융 복지", "금융 취약 계층 지원",
    "금융 서비스 개선", "금융 인프라", "금융 접근 개선", "금융 교육 프로그램",
    "대출 한도 확대", "금리 인하", "보증 한도 확대"
]

def extract_named_entities(text):
    entities = {"공기업": [], "정책": []}
    if not isinstance(text, str):
        return entities
    # 공기업 관련 엔터티 추출
    for company in public_company_list:
        if company in text:
            entities["공기업"].append(company)
    # 정책 관련 엔터티 추출
    for keyword in policy_keywords:
        if keyword in text:
            entities["정책"].append(keyword)
    return entities

# '본문' 컬럼에 대해 추가 전처리 적용
df['본문_정제'] = df['본문'].apply(clean_text)
df['tokens'] = df['본문_정제'].apply(preprocess_text)
df['엔터티'] = df['본문'].apply(extract_named_entities)

# 전처리 결과 일부 확인
print("✅ 추가 전처리 결과 샘플:")
print(df[['제목', 'tokens', '엔터티']].head(), "\n")


# 엔터티에서 "공기업"과 "정책" 키워드가 모두 있는 기사만 선택
filtered_df = df[df['엔터티'].apply(lambda x: bool(x.get("공기업")) and bool(x.get("정책")))]
print("✅ 공기업 + 정책 관련 기사 수:", len(filtered_df))
print(filtered_df[['제목', '엔터티']].head(), "\n")

#저장
final_filename = "/content/최종_필터링_완료.xlsx"
filtered_df.to_excel(final_filename, index=False)
print(f"저장 완료: {final_filename}")
